# Combine all on- and off-shore land and barrier layers
This is an example workflow for setting up the friction and barriers for an
off-shore least cost paths analysis. This incorporates existing land-based costs
and barriers into the final friction and barrier rasters. Note that costs for 
offshore transmission are not actually calculated. Instead, relative frictions 
are assigned to areas that areas that difficult to lay cables in, such as steep 
terrain and shipping lanes. 

Areas that should not be used to transmission are referred to as barriers. Note 
that barriers are not inpenetrable in 

In [2]:
from reVX.least_cost_xmission.aoswt_utilities import CombineRasters

In [3]:
# Default layer location so we don't have to write the full path
layer_dir = '/shared-projects/rev/projects/aoswt/data/exclusions'

# Template raster to pull transform, etc. from
template_f ='/shared-projects/rev/projects/aoswt/data/exclusions/adjusted_shipping_lanes.tif'

In [4]:
# The barriers have one or more values to "bar" in the first position of
# the tuple. Single values should be an int, multiple values should use a list. 
# Any other values in the raster are considered open and not to be a barrier.
barrier_files = [
    (1, 'military_ship_shock_boxes.tif'),
    (1, 'artificial_reefs.tif'),
    (1, 'danger_zones_and_restricted_areas.tif'),
    (1, 'usace_placement_areas.tif'),
    (1, 'usace_sand_borrow_areas.tif'),
    (1, 'usace_sad_sand_borrow_areas.tif'),
    (1, 'boem_sand_borrow_areas.tif'),
    (1, 'boem_marine_mineral_areas.tif'),
    (1, 'unexploded_ordnance_locations_100m_buffer.tif'),
    (1, 'dpp_option_areas.tif'),
    (3, 'dod_designations.tif'),
    ([1,2,3,4], 'ocean_disposal_sites.tif'),
    ([1,2,3,4,6], 'conservation_areas.tif'),
    (1, '/shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif'),
]

# Force include layers. These layers will override the barrier layers. Cells
# with values in the first position of the tuple (int or list) will be forced
# "open" if barricaded by the barrier_files. Cells with any other values will
# not be affected.
fi_files = [
    (1, 'boem_wind_planning_areas_03292021.tif'),
    (1, 'boem_wind_leases_06082021.tif'),
    (1, 'aoswt_ad_hoc_forced_inclusions.tif'),
]

friction_files = [
    # ({'cell value in tiff': 'corresponding friction', ...}, 'filename.tif')
    ({1: 1, 2: 10, 3:5}, 'adjusted_shipping_lanes.tif'),
    ({1: 5}, 'abb_gas_pipelines_61m_buffer.tif'),
    ({1: 10}, 'federal_channels.tif'),
    ({1: 10, 2: 5, 3: 0},'/shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif'),
]

slope_file = 'atlantic_coast_slope.tif'

bathy_file = '/shared-projects/rev/projects/weto/fy21/offshore/data/rasters/keepers/bathymetry.tif'

In [5]:
cr = CombineRasters(template_f, layer_dir=layer_dir)

In [6]:
# Build composite barrier layer. Note
cr.build_off_shore_barriers(barrier_files, fi_files, slope_file=slope_file, 
                            save_tiff=True)

Loading barrier layers
--- military_ship_shock_boxes.tif
--- artificial_reefs.tif
--- danger_zones_and_restricted_areas.tif
--- usace_placement_areas.tif
--- usace_sand_borrow_areas.tif
--- usace_sad_sand_borrow_areas.tif
--- boem_sand_borrow_areas.tif
--- boem_marine_mineral_areas.tif
--- unexploded_ordnance_locations_100m_buffer.tif
--- dpp_option_areas.tif
--- dod_designations.tif
--- ocean_disposal_sites.tif
--- conservation_areas.tif
--- /shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif
--- calculating slope barrier
Building composite off-shore barrier layers
Loading forced inclusion layers
--- boem_wind_planning_areas_03292021.tif
--- boem_wind_leases_06082021.tif
--- aoswt_ad_hoc_forced_inclusions.tif
Building composite forced inclusion layers
Saving barriers to os_barriers.tif
Done building barrier layers


In [7]:
# Build composite friction layer
cr.build_off_shore_friction(friction_files, slope_file=slope_file, 
                            bathy_file=bathy_file,
                            bathy_depth_cutoff=-200, 
                            save_tiff=True)

Loading friction layers
--- setting raster value 1 to fricton 1 for adjusted_shipping_lanes.tif
--- setting raster value 2 to fricton 10 for adjusted_shipping_lanes.tif
--- setting raster value 3 to fricton 5 for adjusted_shipping_lanes.tif
--- setting raster value 1 to fricton 5 for abb_gas_pipelines_61m_buffer.tif
--- setting raster value 1 to fricton 10 for federal_channels.tif
--- setting raster value 1 to fricton 10 for /shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif
--- setting raster value 2 to fricton 5 for /shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif
--- setting raster value 3 to fricton 0 for /shared-projects/rev/projects/aoswt/data/rasters/conmap_sediment_low_medium_high.tif
--- calculating slope friction
Combining off-shore friction layers
Saving combined friction to tiff
Done processing friction layers


In [8]:
# Existing AOSWT h5 file to pull lat/lng, profile from
ex_aoswt_h5 = '/shared-projects/rev/projects/aoswt/data/exclusions/AOSWT_Exclusions.h5'

# New AOSWT h5 file to write costs and barriers to
aoswt_h5 = './example_aoswt_costs.h5'

# Existing land costs and barrier
land_h5 = '/shared-projects/rev/exclusions/xmission_costs.h5'
land_barrier_layer = 'transmission_barrier'
land_costs_layer = 'tie_line_costs_102MW'

In [9]:
# Create new AOSWT h5 file if needed
if False:
    cr.create_aoswt_h5(ex_aoswt_h5, aoswt_h5, overwrite=True)

In [11]:
# Convert coastal boundary shp file to mask if not already done
if False:
    land_shp = '/shared-projects/rev/projects/aoswt/data/shapefiles/coast/gshhs_f_l1_rev.shp'
    cr.create_land_mask(land_shp, save_tiff=True)
else:
    cr.load_land_mask()

Rasterizing /shared-projects/rev/projects/aoswt/data/shapefiles/coast/gshhs_f_l1_rev.shp
Saving land mask to land_mask.tif
Rasterizing complete


In [12]:
# Merge barriers and write to h5
cr.merge_os_and_land_barriers(land_h5, land_barrier_layer, aoswt_h5, save_tiff=True)

Loading land barriers "transmission_barrier" from /shared-projects/rev/exclusions/xmission_costs.h5
Reprojecting land barriers
Combining land and off-shore barriers
Saving combined barriers to combo_barriers.tif
Writing combined data to "transmission_barrier" in /shared-projects/rev/projects/aoswt/data/processing/aoswt_costs.h5


In [14]:
# Merge frictions and write to h5. Note that land costs are being scaled down 
# a predetermined amount so that paths can route over short stretches of land.
cr.merge_os_and_land_friction(land_h5, land_costs_layer, aoswt_h5, 
                              land_cost_mult=1/15000, save_tiff=True) 

Loading land friction "tie_line_costs_102MW" from /shared-projects/rev/exclusions/xmission_costs.h5
Reprojecting land friction
Combining land and off-shore friction
Saving combined friction to combo_friction.tif
Writing combined data to "tie_line_costs_102MW" in /shared-projects/rev/projects/aoswt/data/processing/aoswt_costs.h5
